In [4]:
import pandas as pd
import os

parent_dir = 'data/raw/hotels'
files = [os.path.join(parent_dir, file) for file in os.listdir(parent_dir)]
dfs = [pd.read_csv(file, delimiter=';', encoding='ISO-8859-1') for file in files]
df_hotels = pd.concat(dfs)

df_hotels.shape

(211187, 9)

In [8]:
df_hotels.duplicated(df_hotels.columns[:-2]).sum()

52579

In [10]:
df_hotels.drop_duplicates(df_hotels.columns[:-2], inplace=True)

In [11]:
df_hotels.reset_index(drop=True, inplace=True)

In [12]:
def strip_url(url: str):
    '호텔 url 도로명 주소 부분까지만 strip'
    if isinstance(url, str):
        to_return = url[:url.find('html') + 4]
    else:
        to_return = 'str 아님'
    return to_return

In [13]:
urls = df_hotels['url'].map(strip_url)
urls_unique = urls.unique()

In [26]:
import requests
from bs4 import BeautifulSoup
import json

data = []
for idx, url in enumerate(urls):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    json_str = soup.find('script', type='application/ld+json').text
    json_dict = json.loads(json_str)
    datapoint = pd.DataFrame([[json_dict['address']['addressLocality']], [json_dict['address']['postalCode']]], columns=['local_address', 'postal_code'])
    print(datapoint.values)
    data.append(datapoint)
    if idx == 1:
        break

addresses = pd.concat(data)

ValueError: Shape of passed values is (2, 1), indices imply (2, 2)

In [25]:
pd.concat([[1],[2]])

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [26]:
import json

json_dict = json.loads(json_str)
json_dict

{'priceRange': '다가오는 날짜의 숙박 요금: 1박 기준 ₩79,000부터 (최저가 맞춤)',
 'aggregateRating': {'@type': 'AggregateRating',
  'bestRating': 10,
  'reviewCount': 35,
  'ratingValue': 7.8},
 'address': {'addressLocality': '조천읍 일주동로 1415',
  'addressCountry': '대한민국',
  'addressRegion': '제주도',
  '@type': 'PostalAddress',
  'postalCode': '63339',
  'streetAddress': '제주, 조천읍 일주동로 1415'},
 'description': '제주 필 하우스는 제주시 함덕 해수욕장에서 차로 단 3분 거리에 있습니다. 호텔은 자체 취사형 객실, 무료 Wi-Fi 및 공용 주차장을 갖추고 있습니다. 에어컨과 난방 시설을 갖춘 스튜디오에는 테라스, TV 및 스토브, 냉장고, 전기 주전자가 구비된 주방이 마련되어 있습니다.',
 'image': 'https://cf.bstatic.com/xdata/images/hotel/max500/304419275.jpg?k=8c9c7dd2f7586bcb24306942bf988965da64d2784f58eb384bd430f8a639a3d4&o=&hp=1',
 '@context': 'http://schema.org',
 'name': '제주 필 하우스',
 '@type': 'Hotel',
 'url': 'https://www.booking.com/hotel/kr/jeju-feel-house.ko.html',
 'hasMap': 'https://maps.googleapis.com/maps/api/staticmap?sensor=false&markers=color:blue%7c33.5450610,126.6864395&size=1600x1200&zoom=15&center=33.5450610,126.686

In [27]:
json_dict['address']

{'addressLocality': '조천읍 일주동로 1415',
 'addressCountry': '대한민국',
 'addressRegion': '제주도',
 '@type': 'PostalAddress',
 'postalCode': '63339',
 'streetAddress': '제주, 조천읍 일주동로 1415'}

In [3]:
def strip_url(url: str):
    if isinstance(url, str):
        to_return = url[:url.find('html') + 4]
    else:
        to_return = 'str 아님'
    return to_return

stripped_urls = df['url'].map(strip_url)

In [4]:
stripped_urls.nunique()

231

In [6]:
stripped_urls[0]

'https://www.booking.com/hotel/kr/playce-camp-jeju.ko.html'

In [ ]:
from selenium import webdriver
